# Importar librerías necesarias y definir variables

In [ ]:
import json

from typing import List
import sys
from pathlib import Path
import os
from statistics import mean, stdev

from src.catching import attempt_catch
from src.pokemon import PokemonFactory, StatusEffect, Pokemon

DEFAULT_NOISE = 0  # Dijeron que no lo usemos, salvo para el 2c y no sé si el 2d

OUTPUT_PATH = "output"  # Relativo al src

EJ1_FILENAME = "Ej1.csv" # Relativo al output

EJ2_FILENAME = "Ej2.csv" # Relativo al output

CONFIG_FILENAME = "config.json" # Relativo al src

OUTPUT_HTML_PLOT_NAME = "first_figure.html"

In [ ]:
def get_src() -> Path:
    return Path(get_src_str())


def get_src_str() -> str:
    return os.getcwd() + "/"


def move_to_src() -> None:
    src = get_src()
    os.chdir(src)


def get_output_dir() -> Path:
    return Path(get_src()).joinpath(OUTPUT_PATH)


def get_output_dir_str() -> str:
    return str(get_output_dir().resolve()) + "/"

# Cargar configuración

In [ ]:
class ConfigData:
    iterations: int = 100
    pokeballs: List[str] = ["pokeball", "ultraball", "fastball", "heavyball"]
    pokemon_names: List[str] = ["snorlax"]
    level: int = 100
    status_effect: str = "none"
    health: float = 1.0

def load_config() -> ConfigData:
    config_data = ConfigData()
    if len(sys.argv) == 1:
        return config_data

    with open(get_src().joinpath(CONFIG_FILENAME), "r") as config_f:
        config = json.load(config_f)

        # With default values
        try:
            config_data.iterations = config["iterations"]
        except KeyError:
            pass
        try:
            config_data.pokeballs = config["pokeballs"]
        except KeyError:
            pass
        try:
            config_data.pokemon_names = config["pokemons"]
        except KeyError:
            pass
        try:
            config_data.level = config["level"]
        except KeyError:
            pass
        try:
            config_data.status_effect = config["status_effect"]
        except KeyError:
            pass
        try:
            config_data.health = config["health"]
        except KeyError:
            pass
    return config_data

In [ ]:
config = load_config()

# Ej 1

## Definición de Funciones

In [ ]:
def stress_pokeball(ball: str, pkmons: List[Pokemon], n: int):
    """Tests the given Pokeball with the provided list of Pokemons and returns the average catch rate and the standard
    deviation

    :param ball: The name of the type of ball to use
    :type ball: str
    :param pkmons: List of Pokemons to try catching
    :type pkmons: list[Pokemon]
    :param n: The amount of iterations to test
    :type n: int
    :return:
        The catch rate of the Pokeball in relation to the given Pokemons, as a tuple with the mean catch rate and
        its standard deviation
    """
    catch_rates = []
    catches = 0
    for pkmon in pkmons:
        for _ in range(n):
            caught, rate = attempt_catch(pkmon, ball, DEFAULT_NOISE)
            catch_rates.append(rate)
            catches += 1 if caught else 0
    mean_rate = mean(catch_rates)
    # print(f"min: {min(catch_rates)} || max: {max(catch_rates)} || avg: {mean_rate}")
    return [mean_rate, stdev(catch_rates, mean_rate)]


def create_all_pokemons(names: List[str], lvl: int, status: StatusEffect, health: float) -> List[Pokemon]:
    factory = PokemonFactory("pokemon.json")
    pokemons_to_ret: List[Pokemon] = []
    for pokemon_name in names:
        pokemons_to_ret.append(factory.create(pokemon_name, lvl, status, health))
    return pokemons_to_ret

## Resolución

In [ ]:
pokemons = create_all_pokemons(
        config.pokemon_names,
        config.level,
        StatusEffect.from_value(config.status_effect),
        config.health
    )

output_path = get_output_dir()

os.makedirs(output_path, exist_ok=True)  # create dir if not exists
with open(output_path.joinpath(EJ1_FILENAME), "w") as csv_f:

    csv_f.write("pokeball,avg_prob,stdev\n")

    for pokeball in config.pokeballs:
        poke_prob_avg, poke_stdev = stress_pokeball(pokeball, pokemons, config.iterations)
        print(f"Pokebola: {pokeball}")
        print(f"average_prob: {poke_prob_avg}")
        print(f"deviation: {poke_stdev}")
        print("---------------------")
        csv_f.write(f"{pokeball},{poke_prob_avg},{poke_stdev}\n")

## Gráficos

In [ ]:
import plotly.express as px
import pandas as pd

def plot_values(x: List[any], y: List[float], e_y, title="Title"):
    fig = px.bar(x=x, y=y, title=title, error_y=e_y)
    fig.update_layout(title_font_size=50)
    fig.write_html(get_output_dir().joinpath(OUTPUT_HTML_PLOT_NAME), auto_open=True)

In [ ]:
data_frame = pd.read_csv(get_output_dir().joinpath(EJ1_FILENAME), sep=',')

plot_values(data_frame["pokeball"], data_frame["avg_prob"], data_frame["stdev"], "Pokeball accuracy")

print(data_frame)

## Ej2

## Resolucion

In [ ]:
with open(output_path.joinpath(EJ2_FILENAME), "w") as csv_f:

    csv_f.write("pokeball,pokemon,avg_prob,stdev\n")

    for pokeball in config.pokeballs:
        for pokemon in pokemons:
            poke_prob_avg, poke_stdev = stress_pokeball(pokeball, [pokemon], config.iterations)
            print(f"Pokebola: {pokeball}")
            print(f"Pokemon: {pokemon.name}")
            print(f"average_prob: {poke_prob_avg}")
            print(f"deviation: {poke_stdev}")
            print("---------------------")
            csv_f.write(f"{pokeball},{pokemon.name},{poke_prob_avg},{poke_stdev}\n")

## Graficos

In [ ]:
import plotly.graph_objects as go

def bar_group_plot(pokeballs: List[str], pkmons: List[str], pkmon_avg_probs):
    bars = []
    for pkmon in pkmons:
        bars.append(go.Bar(name=pkmon, x=pokeballs, y=pkmon_avg_probs[pkmon]))

    fig = go.Figure(data=bars)
    # Change the bar mode
    fig.update_layout(barmode='group')
    fig.show()

data_frame = pd.read_csv(get_output_dir().joinpath(EJ2_FILENAME), sep=',')

grouped_data = data_frame.groupby(['pokemon', 'pokeball'])['avg_prob'].mean().unstack()
pokeball_list = grouped_data.columns.tolist()

normalized_avg_probs = {}
for pokemon in data_frame['pokemon'].unique():
    base_avg_prob = grouped_data.loc[pokemon, "pokeball"]
    normalized_avg_probs[pokemon] = [avg_prob / base_avg_prob for avg_prob in grouped_data.loc[pokemon].tolist()]


bar_group_plot(pokeball_list, data_frame['pokemon'].unique(), normalized_avg_probs)
